In [ ]:
import pandas as pd
import os
import shutil
from pathlib import Path
from sklearn.model_selection import train_test_split
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
SAVE_CSV_PATH = '/content/drive/MyDrive/HECTO/Dataset/CSV/celeb_v2_final.csv'
ZIP_PATH = "/content/drive/MyDrive/HECTO/Dataset/final_files/Celeb_frames_pp_final.zip"
LOCAL_EXTRACT_BASE = Path("/content/Celeb_Extract")

In [ ]:
if LOCAL_EXTRACT_BASE.exists():
    shutil.rmtree(LOCAL_EXTRACT_BASE)
LOCAL_EXTRACT_BASE.mkdir(parents=True, exist_ok=True)

print(f"🔓 압축 해제 시작: {ZIP_PATH}")
# -q: quiet, -d: target directory
!unzip -q "{ZIP_PATH}" -d "{LOCAL_EXTRACT_BASE}"
print("✅ 압축 해제 완료!")

CELEB_FRAMES_LOCAL = LOCAL_EXTRACT_BASE / "celeb_frames"

data_list = []
print("🔍 로컬 폴더 스캔 중 (Real=1, Fake=0)...")

🔓 압축 해제 시작: /content/drive/MyDrive/HECTO/Dataset/final_files/Celeb_frames_pp_final.zip
✅ 압축 해제 완료!
🔍 로컬 폴더 스캔 중 (Real=1, Fake=0)...


In [ ]:
data_list = []

# rglob은 하위 폴더를 다 뒤지므로, 로컬 베이스 경로에서 바로 찾습니다.
print("🔍 이미지 파일 스캔 중...")
all_images = list(LOCAL_EXTRACT_BASE.rglob("*.jpg"))

if not all_images:
    print("❌ 파일을 찾지 못했습니다! 압축 해제된 폴더의 실제 경로를 확인하세요.")
    # 실제 어떤 폴더들이 생겼는지 출력해봅니다.
    !ls -R {LOCAL_EXTRACT_BASE} | head -n 20
else:
    for img_path in all_images:
        path_str = str(img_path).lower()

        # 라벨링: Real=1, Fake=0
        label = 1 if 'real' in path_str else 0

        # Video ID 추출 로직 (폴더 구조에 맞춰 유연하게)
        # 보통 이미지의 부모 폴더가 00000 같은 숫자면 그 위 폴더가 video_id
        parent_name = img_path.parent.name
        if parent_name.isdigit() or len(parent_name) < 3:
            video_id = img_path.parent.parent.name
        else:
            video_id = parent_name

        data_list.append({
        'video_id': video_id,
        'frame_path': str(img_path),
        'label': label,
        'source': 'celeb'
    })

    # 데이터프레임 생성
    df = pd.DataFrame(data_list)

    if not df.empty:
        print(f"📊 수집 완료: {df['video_id'].nunique()}개 영상, {len(df)}개 프레임")

        # 4. 8:1:1 분할
        video_meta = df.drop_duplicates('video_id')[['video_id', 'label']]

        train_ids, temp_ids = train_test_split(
            video_meta['video_id'],
            test_size=0.2,
            random_state=42,
            stratify=video_meta['label']
        )

        temp_meta = video_meta[video_meta['video_id'].isin(temp_ids)]
        val_ids, test_ids = train_test_split(
            temp_meta['video_id'],
            test_size=0.5,
            random_state=42,
            stratify=temp_meta['label']
        )

        df['split'] = None
        df.loc[df['video_id'].isin(train_ids), 'split'] = 'train'
        df.loc[df['video_id'].isin(val_ids), 'split'] = 'val'
        df.loc[df['video_id'].isin(test_ids), 'split'] = 'test'

        # 결과 확인 및 저장
        print("\n--- [분할 결과] ---")
        print(df.groupby(['split', 'label']).size().unstack(fill_value=0))

        os.makedirs(os.path.dirname(SAVE_CSV_PATH), exist_ok=True)
        df.to_csv(SAVE_CSV_PATH, index=False)
        print(f"\n🚀 CSV 저장 완료: {SAVE_CSV_PATH}")
    else:
        print("❌ 데이터프레임이 비어있습니다.")

🔍 이미지 파일 스캔 중...
📊 수집 완료: 1116개 영상, 16740개 프레임

--- [분할 결과] ---
label     0     1
split            
test    840   840
train  6690  6690
val     840   840

🚀 CSV 저장 완료: /content/drive/MyDrive/HECTO/Dataset/CSV/celeb_v2_final.csv
